In [ ]:
!pip install numpy

In [1]:
import numpy as np
import math, random

In [33]:
def sigmoid(t):
    return 1 / (1 + math.exp(-t))


def neuronio_MCP(pesos, entradas):
    uk = np.dot(pesos, entradas)
    return sigmoid(uk)


def feed_forward(rede, vetor_entrada):
    vetor_saida = []

    for ponteiro in rede:

        entrada_com_bias = vetor_entrada + [1]
        saida = [neuronio_MCP(neuronio, entrada_com_bias) for neuronio in ponteiro]

        vetor_saida.append(saida)

        vetor_entrada = saida

    return vetor_saida

def backpropagation(rede_neural, vetor_entrada, vetor_saida, alpha):
    saidas_intermediarias, saidas_neuronios = feed_forward(rede_neural, vetor_entrada)
    deltas_saida = [
        saida * (1 - saida) * (saida - vetor_saida[i]) * alpha
        for i, saida in enumerate(saidas_neuronios)
    ]

    for i, neuronio_saida in enumerate(rede_neural[-1]):
        for j, saida_intermediaria in enumerate(saidas_intermediarias + [1]):
            neuronio_saida[j] -= deltas_saida[i] * saida_intermediaria

    deltas_intermediarios = [
        saida_intermediaria
        * (1 - saida_intermediaria)
        * np.dot(deltas_saida, [n[i] for n in rede_neural[-1]])
                
        for i, saida_intermediaria in enumerate(saidas_intermediarias)
    ]

    for i, neuronio_intermediario in enumerate(rede_neural[0]):
        for j, entrada in enumerate(vetor_entrada + [1]):
            neuronio_intermediario[j] -= deltas_intermediarios[i] * entrada

def predict(entrada):
    return feed_forward(rede_neural, entrada)[-1]

In [49]:
def constroi_matriz_de_entrada(lista_de_caracteres):
    return [1 if c == '1' else 0
            for row in lista_de_caracteres.split("\n")
            for c in row.strip()]

In [73]:
lista_treino = [
    """11111
       1...1
       11111
       1...1
       1...1""",

    """11111
       1....
       11111
       1....
       11111""",

    """11111
       ..1..
       ..1..
       ..1..
       11111""",

    """11111
       1...1
       1...1
       1...1
       11111""",

    """1...1
       1...1
       1...1
       1...1
       11111""",
]

In [74]:
caracteres = list(map(constroi_matriz_de_entrada, lista_treino))
print("matriz de entrada:\n", caracteres)

matriz de entrada:
 [[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]]


In [64]:
# Númmero 1 de cada linha indica qual a vogal
# Exs:
# [1, 0, 0, 0, 0] representa o caractere "a"
# [0, 0, 1, 0, 0] representa o caractere "i"
saidas = [[1 if i == j else 0 for i in range(5)]
            for j in range(5)]
print("saidas:\n", saidas)

saidas:
 [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]


In [65]:
random.seed(0)

alpha = 0.99

# Cada caractere de entrada é representado por uma matriz 5x5
dimensao_entrada = 25

neuronios_intermediarios = 5

# Cada caractere de saída é representado por um vetor com 5 posições
dimensao_saida = 5

In [66]:
camada_intermediaria = [[random.random() for _ in range(dimensao_entrada + 1)]
                           for _ in range(neuronios_intermediarios)]

camada_saida = [[random.random() for _ in range(neuronios_intermediarios + 1)]
                           for _ in range(dimensao_saida)]

rede_neural = [camada_intermediaria, camada_saida]
# print("rede neural:\n", rede_neural)

In [67]:
# Treinamento
qtd_ciclos = 10000
for _ in range(qtd_ciclos):
    for vetor_entrada, vetor_saida in zip(caracteres, saidas):
        backpropagation(rede_neural, vetor_entrada, vetor_saida, alpha)

In [68]:
vogais = ["a", "e", "i", "o", "u"]

# Teste com as entradas treinadas
for i, numeros in enumerate(caracteres):
    saidas_rede = predict(numeros)
    print(vogais[i], [round(p, 2) for p in saidas_rede])

a [0.98, 0.01, 0.0, 0.0, 0.01]
e [0.01, 0.98, 0.01, 0.01, 0.0]
i [0.0, 0.01, 0.99, 0.01, 0.0]
o [0.0, 0.01, 0.01, 0.98, 0.01]
u [0.01, 0.0, 0.0, 0.01, 0.99]


In [69]:
# Diferente estilo de caracteres
lista_teste = [
    """.111.
       1...1
       11111
       1...1
       1...1""",

    """.1111
       1....
       1111.
       1....
       .1111""",

    """.....
       .11..
       ..1..
       ..1..
       .111.""",

    """.....
       .111.
       1...1
       1...1
       .111.""",

    """.....
       1...1
       1...1
       1...1
       .111.""",
]

In [70]:
caracteres_diferentes = list(map(constroi_matriz_de_entrada, lista_teste))
# print("matriz de entrada:\n", caracteres_diferentes)

In [71]:
# Teste com entradas diferentes do treinamento 
for i, numeros in enumerate(caracteres_diferentes):
    saidas_rede = predict(numeros)
    print(vogais[i], [round(p, 2) for p in saidas_rede])

a [0.98, 0.01, 0.0, 0.0, 0.01]
e [0.01, 0.98, 0.01, 0.01, 0.0]
i [0.0, 0.0, 0.97, 0.06, 0.0]
o [0.0, 0.0, 0.0, 0.4, 0.4]
u [0.01, 0.0, 0.0, 0.05, 0.93]
